In [1]:
import os
import requests
from pydantic import BaseModel, ValidationError
from typing import Annotated, Literal, Optional
from datetime import datetime
from autogen import ConversableAgent, register_function
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
import numpy as np
import pandas as pd
from io import StringIO
import spacy
from bs4 import BeautifulSoup
import time
import json
import re
from typing import List, Dict, Annotated

load_dotenv()

True

In [2]:
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv('LANGCHAIN_ENDPOINT')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'


In [3]:
def documents_selector(input: Annotated[str, "Qualitative data input prompt"]) -> List[Dict[str, str]]:

    llm = ChatOpenAI(
        model="gpt-4",
        temperature=0.3
    )
    

    # need to come up with idea to remove this line in the prompt: Also make a note that the current financial year is 2024.
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", '''
                Your task is to identify and fill the following entities from the user's prompt:

                1. **ticker**: The abbreviation of the company's name.
                2. **document_type**: 'Form 10K' (for annual financial details), 'Form 10Q' (for quarterly financial details), etc.
                3. **year**: 
                   - If 'Form 10K' is chosen, the year should be in the format 'YYYY' (e.g., 2021, 2019).
                   - If 'Form 10Q' is chosen, the year should be in the format 'YYYY QX' (e.g., 2023 Q3, 2022 Q2).

                User prompts may be explicit or implicit and may not specify all required entities. Use context to infer missing details.

                If a user prompt is complex, you may break it into simpler, atomic prompts and provide separate outputs for each.
             
                Make a note that there is no fourth quarter Q4, companies file for Form 10K instead of filing Form 10Q. So you can only choose Q1, Q2 or Q3.
             
                Also make a note that the current financial year is 2024.

                ### Guidelines for Handling Qualitative Financial Questions:

                1. **Map to Relevant Documents**: Use the most relevant financial documents (annual or quarterly reports) that are likely to contain information on market trends, growth prospects, or other qualitative aspects.

                2. **Use Representative Entities**: When the user question is about a sector or general trends, use representative entities such as sector ETFs (e.g., XLK for technology) to provide the context.

                3. **Infer Contextual Details**: Use the context provided in the user's question to infer missing details and make reasonable assumptions about the tickers, document types and years.

                4. **Use top companies tickers**: If you are stuck which company's ticker to choose then use one or more top performing company's ticker in the respective field.
             
                5. **Use recent years**: If you are stuck with which year to choose then choose the latest year or years or latest quarter based on the context of the user prompt.

                ### Examples:
                1. **Prompt**: 'Please provide the report on recent developments at Apple.'
                   **Output**: 
                     ticker: AAPL
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2023 Q3

                2. **Prompt**: 'Please provide the report on third quarter for Apple for the years 2021 and 2022.'
                   **Output**: 
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2021 Q3
                     
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2022 Q3

                3. **Prompt**: 'What are the prevailing market trends in the technology sector?'
                   **Output**: 
                     ticker: XLK
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: XLK
                     document_type: Form 10Q
                     year: 2023 Q3

                4. **Prompt**: 'What are the growth prospects of Tesla in the next five years?'
                   **Output**: 
                     ticker: TSLA
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: TSLA
                     document_type: Form 10Q
                     year: 2023 Q3

                Return 'TERMINATE' when the task is completed.
            '''),
            ("user", "{input}")
        ]
    )
    
    chain = prompt | llm
    
    output = chain.invoke({"input": input})

    try:
        documents = output.content.split('\n')
        result = []
        current_doc = {}

        for line in documents:
            if 'ticker:' in line:
                if current_doc:
                    result.append(current_doc)
                    current_doc = {}
                current_doc['ticker'] = line.split('ticker:')[1].strip()
            elif 'document_type:' in line:
                current_doc['document_type'] = line.split('document_type:')[1].strip()
            elif 'year:' in line:
                current_doc['year'] = line.split('year:')[1].strip()
        
        if current_doc:  # Add the last document
            result.append(current_doc)

        return result

    except Exception as e:
        raise ValueError(f"Failed to parse output: {output.content}. Error: {e}")

In [4]:
documents_selector('Analyse the performance of PLTR for past two years')

[{'ticker': 'PLTR', 'document_type': 'Form 10K', 'year': '2022'},
 {'ticker': 'PLTR', 'document_type': 'Form 10K', 'year': '2023'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2022 Q1'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2022 Q2'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2022 Q3'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2023 Q1'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2023 Q2'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2023 Q3'}]

In [5]:
documents_selector('Analyse the performance of PLTR and AMZN for past two years')

[{'ticker': 'PLTR', 'document_type': 'Form 10K', 'year': '2022'},
 {'ticker': 'PLTR', 'document_type': 'Form 10K', 'year': '2023'},
 {'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2022'},
 {'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2022 Q3'},
 {'ticker': 'PLTR', 'document_type': 'Form 10Q', 'year': '2023 Q3'},
 {'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2022 Q3'},
 {'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}]

In [6]:
documents_selector("What are the strengths and weakness of NVDA and AMD over the past two years?")

[{'ticker': 'NVDA', 'document_type': 'Form 10K', 'year': '2022'},
 {'ticker': 'NVDA', 'document_type': 'Form 10Q', 'year': '2022 Q3'},
 {'ticker': 'AMD', 'document_type': 'Form 10K', 'year': '2022'},
 {'ticker': 'AMD', 'document_type': 'Form 10Q', 'year': '2022 Q3'},
 {'ticker': 'NVDA', 'document_type': 'Form 10K', 'year': '2023'},
 {'ticker': 'NVDA', 'document_type': 'Form 10Q', 'year': '2023 Q3'},
 {'ticker': 'AMD', 'document_type': 'Form 10K', 'year': '2023'},
 {'ticker': 'AMD', 'document_type': 'Form 10Q', 'year': '2023 Q3'}]

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
prompt_rag_fusion 
| ChatOpenAI(temperature=0) 
| StrOutputParser() 
| (lambda x: x.split("\n"))
)

# Define the pipeline using LangChain expression language
pipeline = generate_queries | (lambda queries: [{"query": query, "documents": documents_selector(query)} for query in queries])

# Function to run the entire pipeline
def run_pipeline(user_input: str) -> List[Dict[str, List[Dict[str, str]]]]:
    result_documents = pipeline.invoke({"question": user_input})
    return result_documents

# Example usage
user_input = "What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
result_documents = run_pipeline(user_input)

# Output the results
for result in result_documents:
    print(f"Query: {result['query']}")
    for doc in result['documents']:
        print(doc)
    print("---"*30)

Query: 1. What are the biggest cybersecurity risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------
Query: 2. What are the biggest regulatory risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------
Query: 3. What are the biggest competition risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------
Query: 4. What are the biggest supply chain risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'docu

In [8]:
#!/usr/bin/env python
import os
import json
import re
import pdfkit
import logging
from datetime import datetime

try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

import certifi

logging.basicConfig(level=logging.INFO)

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def is_within_quarter(date_str, year, start_month, end_month):
    date_parts = date_str.split('-')
    date_year = date_parts[0]
    date_month = int(date_parts[1])
    return date_year == year and start_month <= date_month <= end_month

def which_fiscal_year(date_str, fiscal_year):
    filing_date = datetime.strptime(date_str.split()[0], "%Y-%m-%d")
    filing_month = filing_date.month
    fiscal_year = int(fiscal_year)

    if (filing_month <= 3):
        fiscal_year += 1
    
    return str(fiscal_year)


def document_downloader(pipeline_output):
    api_key = os.getenv('FMP_API_KEY')
    
    if api_key is None:
        raise ValueError("No API key found. Please set the FMP_API_KEY environment variable.")

    document_type_mapping = {
        "Form 10K": "10-k",
        "Form 10Q": "10-q"
    }

    quarter_map = {
        "Q1": (1, 3),
        "Q2": (4, 6),
        "Q3": (7, 9),
        "Q4": (10, 12)
    }

    filing_type = document_type_mapping.get(pipeline_output.get('document_type'))
    if not filing_type:
        raise ValueError(f"Unsupported document type: {pipeline_output.get('document_type')}")

    ticker = pipeline_output.get('ticker')
    if not ticker:
        raise ValueError("Ticker is required.")

    year_quarter = pipeline_output.get('year')
    year_star = pipeline_output.get('year')
    if not year_quarter:
        raise ValueError("Year/Quarter is required.")

    url = f"https://financialmodelingprep.com/api/v3/sec_filings/{ticker}?type={filing_type}&page=0&apikey={api_key}"

    try:
        fmp_results = get_jsonparsed_data(url)
    except Exception as e:
        raise RuntimeError(f"Failed to fetch data from FMP: {e}")

    if filing_type == "10-k":
        year_quarter = which_fiscal_year(fmp_results[0]['fillingDate'],year_quarter)
        filtered_fmp_results = [i for i in fmp_results if re.search(year_quarter, i['fillingDate'])]
        print(filtered_fmp_results)
        year = year_star
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year)
    elif filing_type == "10-q":
        year, quarter = year_quarter.split()
        start_month, end_month = quarter_map[quarter]
        filtered_fmp_results = [i for i in fmp_results if is_within_quarter(i['fillingDate'], year, start_month, end_month)]
        print(filtered_fmp_results)
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year, quarter)

    if not filtered_fmp_results:
        raise ValueError("No matching SEC filings found.")

    os.makedirs(output_dir, exist_ok=True)

    path_to_wkhtmltopdf = os.getenv('WKHTMLTOPDF_PATH', r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe')
    if not os.path.exists(path_to_wkhtmltopdf):
        raise FileNotFoundError(f"wkhtmltopdf not found at {path_to_wkhtmltopdf}")

    url = filtered_fmp_results[0]['finalLink']
    pdf_path = os.path.join(output_dir, 'primary_document.pdf')

    config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

    try:
        pdfkit.from_url(url, pdf_path, configuration=config)
        logging.info(f"PDF generated and saved at {pdf_path}")
    except Exception as e:
        logging.error(f"PDF generation failed: {e}")
        raise




In [9]:
def check_document_exists(pipeline_output):
    document_type_mapping = {
        "Form 10K": "10-k",
        "Form 10Q": "10-q"
    }

    quarter_map = {
        "Q1": (1, 3),
        "Q2": (4, 6),
        "Q3": (7, 9),
        "Q4": (10, 12)
    }

    filing_type = document_type_mapping.get(pipeline_output.get('document_type'))
    if not filing_type:
        raise ValueError(f"Unsupported document type: {pipeline_output.get('document_type')}")

    ticker = pipeline_output.get('ticker')
    if not ticker:
        raise ValueError("Ticker is required.")

    year_quarter = pipeline_output.get('year')
    if not year_quarter:
        raise ValueError("Year/Quarter is required.")
    
    if filing_type == "10-k":
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year_quarter)
    elif filing_type == "10-q":
        year, quarter = year_quarter.split()
        start_month, end_month = quarter_map[quarter]
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year, quarter)

    pdf_path = os.path.join(output_dir, 'primary_document.pdf')

    if os.path.exists(pdf_path):
        logging.info(f"PDF already exists at {pdf_path}. Skipping download.")
        return True
    
    return False

In [10]:
import requests
import logging

# Define the URL and file path
url = 'https://www.sec.gov/Archives/edgar/data/1018724/000101872424000008/amzn-20231231.htm'
html_path = 'data/sec_filing_test.html'

# Define headers
headers = {
    'User-Agent': 'Traderware/x.tan@traderverse.io'
}

try:
    # Send GET request to the URL with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    # Save HTML content to file
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    
    logging.info(f"HTML generated and saved at {html_path}")

except requests.RequestException as e:
    logging.error(f"Error occurred while downloading the filing: {e}")


INFO:root:HTML generated and saved at data/sec_filing_test.html


In [11]:
for result in result_documents:
    print(f"Query: {result['query']}")
    for doc in result['documents']:
        print(doc)
        if not check_document_exists(doc):
            document_downloader(doc)
    print("---"*30)

C:\Users\goldr\AppData\Local\Temp\ipykernel_21508\2779484452.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Query: 1. What are the biggest cybersecurity risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
[{'symbol': 'AMZN', 'fillingDate': '2024-02-02 00:00:00', 'acceptedDate': '2024-02-01 18:48:30', 'cik': '0001018724', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/1018724/000101872424000008/0001018724-24-000008-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1018724/000101872424000008/amzn-20231231.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\AMZN\10-k\2023\primary_document.pdf


{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
[{'symbol': 'AMZN', 'fillingDate': '2023-08-04 00:00:00', 'acceptedDate': '2023-08-03 18:24:35', 'cik': '0001018724', 'type': '10-Q', 'link': 'https://www.sec.gov/Archives/edgar/data/1018724/000101872423000012/0001018724-23-000012-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1018724/000101872423000012/amzn-20230630.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\AMZN\10-q\2023\Q3\primary_document.pdf
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-k\2023\primary_document.pdf. Skipping download.
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-q\2023\Q3\primary_document.pdf. Skipping download.
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-k\2023\primary_document.pdf. Skipping download.
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-q\2023\Q3\primary_document.pdf. Skipping download.
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-k\2023\primary_document.pdf. Skipping download.
INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-q\2023\Q3\primary_document.pdf. Skipping download.


------------------------------------------------------------------------------------------
Query: 2. What are the biggest regulatory risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------
Query: 3. What are the biggest competition risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------
Query: 4. What are the biggest supply chain risks for Amazon.com Inc. (AMZN)?
{'ticker': 'AMZN', 'document_type': 'Form 10K', 'year': '2023'}
{'ticker': 'AMZN', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
------------------------------------------------------------------------------------------


In [12]:
# Example usage
user_input = "What are the strengths and weakness of NVDA and AMD for the years 2022 and 2023?"
result_documents = run_pipeline(user_input)

for result in result_documents:
    print(f"Query: {result['query']}")
    for doc in result['documents']:
        print(doc)
        if not check_document_exists(doc):
            document_downloader(doc)
    print("---"*30)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
C:\Users\goldr\AppData\Local\Temp\ipykernel_21508\2779484452.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Query: 1. "NVDA strengths and weaknesses 2022"
{'ticker': 'NVDA', 'document_type': 'Form 10K', 'year': '2022'}
[{'symbol': 'NVDA', 'fillingDate': '2023-02-24 00:00:00', 'acceptedDate': '2023-02-24 17:23:43', 'cik': '0001045810', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581023000017/0001045810-23-000017-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581023000017/nvda-20230129.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\NVDA\10-k\2022\primary_document.pdf


{'ticker': 'NVDA', 'document_type': 'Form 10Q', 'year': '2022 Q3'}
[{'symbol': 'NVDA', 'fillingDate': '2022-08-31 00:00:00', 'acceptedDate': '2022-08-31 17:01:35', 'cik': '0001045810', 'type': '10-Q', 'link': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581022000147/0001045810-22-000147-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581022000147/nvda-20220731.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\NVDA\10-q\2022\Q3\primary_document.pdf


...

------------------------------------------------------------------------------------------
Query: 4. "AMD strengths and weaknesses 2023"
{'ticker': 'AMD', 'document_type': 'Form 10K', 'year': '2023'}
[{'symbol': 'AMD', 'fillingDate': '2024-01-31 00:00:00', 'acceptedDate': '2024-01-31 17:16:39', 'cik': '0000002488', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/2488/000000248824000012/0000002488-24-000012-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/2488/000000248824000012/amd-20231230.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\AMD\10-k\2023\primary_document.pdf


{'ticker': 'AMD', 'document_type': 'Form 10Q', 'year': '2023 Q3'}
[{'symbol': 'AMD', 'fillingDate': '2023-08-02 00:00:00', 'acceptedDate': '2023-08-02 17:12:01', 'cik': '0000002488', 'type': '10-Q', 'link': 'https://www.sec.gov/Archives/edgar/data/2488/000000248823000139/0000002488-23-000139-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/2488/000000248823000139/amd-20230701.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\AMD\10-q\2023\Q3\primary_document.pdf


------------------------------------------------------------------------------------------


In [15]:
document_downloader({'ticker': 'NVDA', 'document_type': 'Form 10Q', 'year': '2006 Q2'})

C:\Users\goldr\AppData\Local\Temp\ipykernel_21508\2779484452.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'NVDA', 'fillingDate': '2006-05-31 00:00:00', 'acceptedDate': '2006-05-31 17:01:49', 'cik': '0001045810', 'type': '10-Q', 'link': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581006000023/0001045810-06-000023-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581006000023/q107form10q.htm'}]


INFO:root:PDF generated and saved at data\sec-edgar-filings\NVDA\10-q\2006\Q2\primary_document.pdf
